In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [8]:
%run helpers.py
image = cv2.imread('C:\\Users\\Lenovo\\Desktop\\solidWhiteCurve.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image) 

Exception: File `'helpers.py'` not found.

In [ ]:
# Same pipeline again. More on Canny and Hough

CANNY_THRESHOLD_1 = 40  # try: 50 - 100      # Typical: 50
CANNY_THRESHOLD_2 = 50 # try: 100 - 200     # Typical: 150
def canny_edges(gray): # https://docs.opencv.org/3.4/d7/de1/tutorial_js_canny.html
    kernel_size = 5 #kernel size for Gaussian smoothing / blurring
    blur_gray= cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)#Gaussian smoothing / blurring

    # finding edges - Canny Edge detection (strong gradient between adjacent pixels)
    edges = cv2.Canny(blur_gray, CANNY_THRESHOLD_1, CANNY_THRESHOLD_2)
    return(edges)


#HOUGH LINES PARAMETERS
RHO = 1                 # try: 1 - 4 (0.5 increments)  
THETA = np.pi/180       # Usually this is Ok
MIN_VOTES = 10          # try: 10 - 50                  # Typical: 30
MIN_LINE_LEN = 5 
MAX_LINE_GAP = 25 

def hough_lines(img):
    lines = cv2.HoughLinesP(img, RHO, THETA, MIN_VOTES, np.array([]), minLineLength=MIN_LINE_LEN, maxLineGap=MAX_LINE_GAP)
    return lines

# Read the image:
image = cv2.imread('test_images/solidWhiteCurve.jpg') #read in
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)#grayscale conversion

# 5.   Get the edges (Canny)
edges_img = canny_edges(gray_img)

# 6.   Remove unwanted parts of the image (too many edges) ROI
ROI_edges_img = helpers_masked_edges(edges_img)

# 7.   Get the lines (from edges - Hough lines)
lines = hough_lines(ROI_edges_img)
hough_lines_image = helpers_draw_lines(lines, ROI_edges_img)

# 8.   Formulate 2 lane lines
lanes = helpers_formulate_lanes(lines, ROI_edges_img)
print(lines.shape)
print(lanes.shape)

# 9.   Create an image with those 2 lines
lanes_image = helpers_draw_lines(lanes, ROI_edges_img)

#lanes_image = np.zeros((ROI_edges_img.shape[0], ROI_edges_img.shape[1], 3), dtype=np.uint8)
#draw_lines(lanes_image, lines)


# 10.   Combine the orginal frame with the 2 lines
final_image = cv2.addWeighted(image, 0.8, lanes_image, 1, 0) 


# Now disply the different steps
fig, (  (image_plt, gray_plt), 
        (edges_plt, ROI_edges_plt), 
        (hough_lines_image_plt, lanes_image_plt),
        (finale_image_plt, finale_image_plt_2)) = plt.subplots(4, 2, figsize=(20, 20))

image_plt.imshow(image)
image_plt.set_title('Original')
gray_plt.imshow(gray_img, cmap='gray')
gray_plt.set_title('Gray')

edges_plt.imshow(edges_img)
edges_plt.set_title('Edges')
ROI_edges_plt.imshow(ROI_edges_img)
ROI_edges_plt.set_title('ROI Edges')

hough_lines_image_plt.imshow(hough_lines_image)
hough_lines_image_plt.set_title('Lines from Hough')
lanes_image_plt.imshow(lanes_image)
lanes_image_plt.set_title('Lanes - 2 lines')

finale_image_plt.imshow(final_image)
finale_image_plt.set_title('Fianl Image')

finale_image_plt_2.imshow(final_image)
finale_image_plt_2.set_title('Fianl Image')

fig.suptitle('Lane lines pipeline')

plt.show()